In [122]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import surprise
import io 
from surprise import Reader, Dataset, SVD, evaluate
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict
from surprise.model_selection import KFold
from surprise import KNNBaseline
from surprise import Dataset
from surprise import get_dataset_dir
from surprise import KNNBasic

import warnings; warnings.simplefilter('ignore')
sns.set_style("ticks")

In [129]:
ratings = pd.read_csv(r'C:\Users\hiren\Desktop\DMProj\ratings_small.csv')
ratings.head(5)


,user_id,movie_id,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [130]:
print("Null Values in Data : ", sum(ratings.isnull().any()))

Null Values in Data :  0


In [131]:

print("\nNumber of ratings :",ratings.shape[0])
print("Number No of Users   :", len(np.unique(ratings.user_id)))
print("Number No of Movie  :", len(np.unique(ratings.movie_id)))


Number of ratings : 100004
Number No of Users   : 671
Number No of Movie  : 9066


In [133]:
f = ['count','mean']

movie_summary = ratings.groupby('movie_id')['rating'].agg(f)
movie_summary.index = movie_summary.index.map(int)
movie_min = round(movie_summary['count'].quantile(0.8),0)
drop_movie_list = movie_summary[movie_summary['count'] < movie_min].index

print('Least Review count for a movie: {}'.format(movie_min))

df_cust_summary = ratings.groupby('user_id')['rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_min = round(df_cust_summary['count'].quantile(0.8),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_min].index


print('Least Review count for a customer: {}'.format(cust_min))


Least Review count for a movie: 13.0
Least Review count for a customer: 202.0


In [134]:
print('Original Data: {}'.format(ratings.shape))
df = ratings[~ratings['movie_id'].isin(drop_movie_list)]
df = ratings[~ratings['user_id'].isin(drop_cust_list)]
print('After Trim Data: {}'.format(df.shape))

print(df.head())

Original Data: (100004, 4)
After Trim Data: (62916, 4)
     user_id  movie_id  rating  timestamp
147        4        10     4.0  949810645
148        4        34     5.0  949919556
149        4       112     5.0  949810582
150        4       141     5.0  949919681
151        4       153     4.0  949811346


In [90]:
df.head()

,user_id,movie_id,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


In [91]:
movie_data = pd.read_csv(r'C:\Users\hiren\Desktop\DMProj\movies_metadata.csv')

In [92]:
movie_data.head()

,adult,belongs_to_collection,budget,genres,homepage,movie_id,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,1,862,tt0114709,en,Toy Story,...,10/30/1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,2,8844,tt0113497,en,Jumanji,...,12/15/1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,3,15602,tt0113228,en,Grumpier Old Men,...,12/22/1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,FALSE,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,4,31357,tt0114885,en,Waiting to Exhale,...,12/22/1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,FALSE,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,5,11862,tt0113041,en,Father of the Bride Part II,...,02-10-1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [93]:
movie_data.set_index('movie_id', inplace = True)
print (movie_data.head(10))

          adult                              belongs_to_collection    budget  \
movie_id                                                                       
1         FALSE  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
2         FALSE                                                NaN  65000000   
3         FALSE  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
4         FALSE                                                NaN  16000000   
5         FALSE  {'id': 96871, 'name': 'Father of the Bride Col...         0   
6         FALSE                                                NaN  60000000   
7         FALSE                                                NaN  58000000   
8         FALSE                                                NaN         0   
9         FALSE                                                NaN  35000000   
10        FALSE  {'id': 645, 'name': 'James Bond Collection', '...  58000000   

                                       

In [94]:
reader = Reader()

data = Dataset.load_from_df(df[['user_id', 'movie_id', 'rating']], reader)
data.split(n_folds=4)

evaluate(SVD(), data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.8917
MAE:  0.6886
------------
Fold 2
RMSE: 0.8942
MAE:  0.6940
------------
Fold 3
RMSE: 0.8837
MAE:  0.6827
------------
Fold 4
RMSE: 0.8925
MAE:  0.6911
------------
------------
Mean RMSE: 0.8905
Mean MAE : 0.6891
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.8917186257618013,
                             0.8942394341125257,
                             0.8836623441522722,
                             0.8924625537448342],
                            'mae': [0.6885654407331936,
                             0.6939730185570748,
                             0.682742241971056,
                             0.6911201870938488]})

In [95]:
n = 4
df_n = df[(df['user_id'] == n) & (df['rating'] >4)]
df_n = df_n.set_index('movie_id')
nm = movie_data['title']
df_n = df_n.join(nm)
print(df_n)

          user_id  rating  timestamp                            title
movie_id                                                             
34              4     5.0  949919556                             Babe
112             4     5.0  949810582              Rumble in the Bronx
141             4     5.0  949919681                     The Birdcage
260             4     5.0  949779042                     Little Women
296             4     5.0  949895708                          Panther
349             4     5.0  949810582                       The Client
356             4     5.0  949919763      Four Weddings and a Funeral
357             4     5.0  949919681                  Higher Learning
364             4     5.0  949949538                    Little Buddha
480             4     5.0  949810582                       Kalifornia
541             4     5.0  949779091                       Son in Law
588             4     5.0  949949486       Terminator 2: Judgment Day
589             4   

In [96]:
reader = Reader()
user_n = movie_data.copy()
user_n = user_n.reset_index()
svd = SVD()
data = Dataset.load_from_df(df[['user_id', 'movie_id', 'rating']], reader)

trainset = data.build_full_trainset()
model = svd.train(trainset)

In [97]:
user_n['Estimate_Score'] = df['movie_id'].apply(lambda x: svd.predict(n, x).est)

user_n = user_n.sort_values('Estimate_Score', ascending=False)
print(user_n.head(10))

       movie_id  adult belongs_to_collection   budget  \
22445    107106  FALSE                   NaN        0   
5193       5289  FALSE                   NaN        0   
37157    153426  FALSE                   NaN        0   
37161    153434  FALSE                   NaN        0   
26391    121366  FALSE                   NaN        0   
26402    121409  FALSE                   NaN        0   
42169    167360  FALSE                   NaN  5000000   
18777     93136  FALSE                   NaN        0   
10783     42952  FALSE                   NaN        0   
15362     78105  FALSE                   NaN        0   

                                                  genres  \
22445                     [{'id': 35, 'name': 'Comedy'}]   
5193                       [{'id': 18, 'name': 'Drama'}]   
37157  [{'id': 16, 'name': 'Animation'}, {'id': 80, '...   
37161  [{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...   
26391  [{'id': 27, 'name': 'Horror'}, {'id': 10769, '...   
26402       

In [98]:
trainset, testset = train_test_split(data, test_size=.25)

svd.fit(trainset)
predictions = svd.test(testset)

In [99]:
print(accuracy.rmse(predictions))
print(accuracy.mae(predictions))

RMSE: 0.8964
0.8963609259523336
MAE:  0.6945
0.6944733041873032


In [104]:
def get_top_n(predictions, n=10):

    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(predictions, n=10)

for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

624 [1213, 58559, 4993, 296, 858, 106920, 1265, 2918, 1193, 1214]
574 [858, 296, 6016, 2959, 2571, 4226, 1196, 4011, 1704, 2716]
577 [4993, 2571, 1222, 541, 593, 8961, 590, 4027, 110, 1282]
452 [260, 1247, 904, 1267, 750, 899, 1266, 2973, 1206, 3088]
652 [2318, 588, 2268, 628, 2096, 6333, 3952, 84152, 1845, 1265]
240 [608, 318, 593, 898, 1617, 1207, 527, 1719, 7153, 8665]
242 [923, 1207, 608, 296, 1213, 593, 903, 1252, 898, 2289]
346 [1073, 8961, 1036, 44191, 2797, 3175, 8874, 1243, 2762, 5040]
285 [1136, 858, 2804, 1196, 1197, 2005, 1089, 2395, 2918, 968]
596 [608, 318, 47, 1213, 1219, 953, 541, 50, 1617, 2028]
56 [1954, 1213, 50, 778, 2000, 80463, 3034, 1208, 4993, 2959]
564 [858, 1198, 1258, 2160, 1079, 608, 1272, 260, 175, 924]
423 [608, 1206, 82459, 4993, 1258, 1213, 4406, 2144, 3168, 55820]
213 [7153, 6539, 356, 5621, 35957, 8368, 97752, 59315, 1032, 13]
353 [2959, 2858, 908, 2542, 1210, 1201, 47, 589, 1387, 4993]
514 [47, 908, 296, 1196, 1193, 1213, 1288, 1258, 1207, 293]
430 [2

In [105]:
kf = KFold(n_splits=5)

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        user_ratings.sort(key=lambda x: x[0], reverse=True)

        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1
    return precisions, recalls


for trainset, testset in kf.split(data):
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

print("Precision:", sum(prec for prec in precisions.values()) / len(precisions))
print("Recall:", sum(rec for rec in recalls.values()) / len(recalls))

Precision: 0.8264197530864191
Recall: 0.06930278600694025


In [106]:
sim_options = {'title': 'pearson', 'user_based': False}
knn = KNNBasic(sim_options=sim_options)
knn.fit(trainset)
predictions = knn.test(testset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [107]:
print(accuracy.rmse(predictions))
print(accuracy.mae(predictions))

RMSE: 0.9222
0.9221863497180887
MAE:  0.7131
0.7130873182022549


In [108]:
movie_data = pd.read_csv(r'C:\Users\hiren\Desktop\DMProj\movies_metadata.csv')
neighbors = knn.get_neighbors(147, k=10)
neighbors
print('The 10 nearest neighbors of An Awfully Big Adventure are:')


for i in neighbors:
    for j in movie_data['movie_id']:
       if j== i:
            print(movie_data['title'][j])


The 10 nearest neighbors of An Awfully Big Adventure are:
Tom and Huck
Sense and Sensibility
Dead Man Walking
Mortal Kombat
Kicking and Screaming
Big Bully
The Crossing Guard
Antonia's Line
Once Upon a Time... When We Were Colored
Shopping


In [109]:
for trainset, testset in kf.split(data):
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=3.5)

print("Precision:", sum(prec for prec in precisions.values()) / len(precisions))
print("Recall:", sum(rec for rec in recalls.values()) / len(recalls))

Precision: 0.8703703703703701
Recall: 0.047204487515625296


In [135]:
sim_options = {'title': 'pearson_baseline', 'user_based': False}
bsl_options = {'method': 'sgd',
               'learning_rate': .005,
               }
knn_b = KNNBaseline(bsl_options=bsl_options, sim_options=sim_options)
knn_b.fit(trainset)
predictions_b = knn_b.test(testset)

Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [136]:
print(accuracy.rmse(predictions_b))
print(accuracy.mae(predictions_b))

RMSE: 0.8752
0.8751998569400411
MAE:  0.6744
0.6743533060452755


In [137]:
neighbors_b = knn_b.get_neighbors(147, k=10)
print()
print('The 10 nearest neighbors of An Awfully Big Adventure are:')


for i in neighbors_b:
    for j in movie_data['movie_id']:
        if j== i:
            print(movie_data['title'][j])


The 10 nearest neighbors of An Awfully Big Adventure are:
The Confessional
Last Summer in the Hamptons
Anne Frank Remembered
Belle de Jour
Canadian Bacon
The Doom Generation
Burnt by the Sun
French Kiss
The Sum of Us
S.F.W.


In [138]:
for trainset, testset in kf.split(data):
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=3.5)

    # Precision and recall can then be averaged over all users
print("Precision:", sum(prec for prec in precisions.values()) / len(precisions))
print("Recall:", sum(rec for rec in recalls.values()) / len(recalls))

Precision: 0.8461728395061723
Recall: 0.0801699030061216
